# A tutorial that shows some features of the Temporal Memory

This program demonstrates some basic properties of the TM, in particular how it handles high-order sequences.

In [ ]:
import numpy as np
import random
random.seed(1)
import matplotlib
import matplotlib.pyplot as plt
from htm.bindings.sdr import SDR
from htm.algorithms import TemporalMemory as TM

%matplotlib inline

## Creating the Temporal Memory

In [ ]:
tm = TM(
    columnDimensions = (2048,),
    cellsPerColumn=8,
    initialPermanence=0.21,
    connectedPermanence=0.3,
    minThreshold=15,
    maxNewSynapseCount=40,
    permanenceIncrement=0.1,
    permanenceDecrement=0.1,
    activationThreshold=15,
    predictedSegmentDecrement=0.01,
)
tm.printParameters()

In [ ]:
sparsity     = 0.02
sparseCols = int(tm.numberOfColumns() * sparsity)
sparseCols

## Create input

We will create a sparse representation of characters A, B, C, D, X, and Y.  
In this particular example we manually construct them, but usually you would use the spatial pooler to build these.

In [ ]:
dataset = {inp : SDR( tm.numberOfColumns() ) for inp in "ABCDXY"}
for i, inp in enumerate("ABCDXY"):
    fr, to = i * sparseCols, (i + 1) * sparseCols
    dataset[inp].dense[fr:to] = 1
    dataset[inp].dense = dataset[inp].dense # This line notifies the SDR that it's dense data has changed in-place.
    print(f"Input {inp} is bits at indices: [{fr} - {to})")

## Aux functions for training and predictions

In [ ]:
def trainTM(sequence, iterations, noiseLevel):
    """
    Trains the TM with given sequence for a given number of time steps and level
    of input corruption

    Parameters
    ----------
    sequence: string
        Sequence of input characters.
    iterations: int
        Number of time TM will be presented with sequence.
    noiseLevel: float
        Amount of noise to be applied on the characters in the sequence.

    Returns
    -------
    x: is list of timestamps / step numbers
    y: is list of prediction accuracy at each step
    """
    ts = 0
    x = []
    y = []
    for t in range(iterations):
        tm.reset()
        for inp in sequence:
            v = SDR(dataset[inp]).addNoise(noiseLevel)
            tm.compute(v, learn=True)
            
            x.append(ts)
            y.append(1 - tm.anomaly)
            ts += 1
    return x, y


def showPredictions():
    """
    Shows predictions of the TM when presented with the characters A, B, C, D, X, and
    Y without any contextual information, that is, not embedded within a sequence.
    """
    
    def _get_printable_ranges(a):
        i = 0
        prints = []
        while i < len(a):
            start = i
            i += 1
            while i < len(a) and a[i-1] + 1 == a[i]:
                i += 1

            if start - i == 1:
                pr = f'{a[start]}'
            else:
                pr = f'{a[start]}-{a[i-1]+1}'
            prints.append(pr)
        
        return str.join(' ', prints)
    
    for inp in sorted(dataset.keys()):
        print("--- " + inp + " ---")
        sdr = dataset[inp]
        tm.reset()
        tm.compute(sdr, learn=False)
        tm.activateDendrites(learn=False)
        activeColumnsIndices = [tm.columnForCell(i) for i in tm.getActiveCells().sparse]
        predictedColumnIndices = [tm.columnForCell(i) for i in tm.getPredictiveCells().sparse]
        print("Active cols: " + _get_printable_ranges(sorted(set(activeColumnsIndices))))
        print("Predicted cols: " + _get_printable_ranges(sorted(set(predictedColumnIndices))))
        print()

## Part 1. Present the sequence ABCD to the TM

The TM will eventually learn the sequence and predict the upcoming characters. This can be measured by the prediction accuracy in plot.  
__N.B__. In-between sequences the prediction accuracy is 0.0 as the TM does not output any prediction.

In [ ]:
def plot_prediction_accuracy(x, y, title):
    plt.ylim([-0.1,1.1])
    plt.plot(x, y)
    plt.xlabel("Timestep")
    plt.ylabel("Prediction Accuracy")
    plt.title(title)
    plt.show()


seqT = "ABCDXY"

seq1 = "ABCD"
x, y = trainTM(seq1, iterations=10, noiseLevel=0.0)
plot_prediction_accuracy(x, y, "Fig. 1: TM learns sequence ABCD")

## Show predictions without context

Once the TM has learned the sequence ABCD, we will present the individual characters to the TM to know its prediction.  
The TM outputs the columns that become active upon the presentation of a particular character as well as the columns predicted in the next time step.

Here, you should see that A predicts B, B predicts C, C predicts D, and D does not output any prediction.  
__N.B__. Here, we are presenting individual characters, that is, a character deprived of context in a sequence. There is no prediction for characters X and Y as we have not presented them to the TM in any sequence. 

In [ ]:
showPredictions()

## Part 2. Present the sequence XBCY to the TM

As expected, the accuracy will drop until the TM learns the new sequence (see plot).  
What would be the prediction of the TM if presented with the sequence BC? This would depend on what character anteceding B. This is an important feature of high-order sequences.

In [ ]:
seq2 = "XBCY"
x, y = trainTM(seq2, iterations=10, noiseLevel=0.0)

# In this figure you can see how the TM starts making good predictions for particular
# characters (spikes in the plot). Then, it will get half of its predictions right, which
# correspond to the times in which is presented with character C. After some time, it
# will learn correctly the sequence XBCY, and predict its characters accordingly.
plot_prediction_accuracy(x, y, "Fig. 2: TM learns new sequence XBCY")

We will present again each of the characters individually to the TM, that is, not within any of the two sequences.

When presented with character A the TM predicts B, B predicts C, but this time C outputs a simultaneous prediction of both D and Y. In order to disambiguate, the TM would require to know if the preceding characters were AB or XB. When presented with character X the TM predicts B, whereas Y and D yield no prediction.

In [ ]:
showPredictions()

## Part 3. Present noisy inputs to the TM.

We would like to see how the TM responds to the presence of noise and how it recovers from it.

We will add noise to the sequence XBCY by corrupting 30% of the bits in the SDR encoding each character. We would expect to see a decrease in prediction accuracy as the TM is unable to learn the random noise in the input (see plot). However, this decrease is not significant.

In [ ]:
x, y = trainTM(seq2, iterations=50, noiseLevel=0.3)
plot_prediction_accuracy(x, y, "Fig. 3: Accuracy in TM with 30% noise in input")

Let's have a look again at the output of the TM when presented with noisy input (30%).  
Here, the noise is low enough that the TM is not affected by it, which would be the case if we saw 'noisy' columns being predicted when presented with individual characters. Thus, we could say that the TM exhibits resilience to noise in its input.

In [ ]:
showPredictions()

Now, we will increase the noise to 60% of the bits in the characters.  
As expected, the predictive accuracy decreases (see plot) and 'noisy' columns are predicted by the TM.

In [ ]:
x, y = trainTM(seq2, iterations=50, noiseLevel=0.6)
plot_prediction_accuracy(x, y, "Fig. 4: Accuracy in TM with 60% noise in input")

In [ ]:
showPredictions()

## Will the TM be able to forget the 'noisy' columns learned in the previous step?

We will present the TM with the original sequence XBCY so it forgets the 'noisy' columns.  
After presenting the uncorrupted sequence XBCY to the TM, we would expect to see the predicted noisy columns from the previous step disappear and the prediction accuracy return to normal. (see plot)

In [ ]:
x, y = trainTM(seq2, iterations=10, noiseLevel=0.0)
plot_prediction_accuracy(x, y, "Fig. 5: When noise is suspended, accuracy is restored")

## Part 4. Present both sequences ABCD and XBCY randomly to the TM.

Here, we might observe simultaneous predictions occurring when the TM is presented with characters D, Y, and C.

For this purpose we will use a blank TM.  
__NB__. Here we will not reset the TM after presenting each sequence with the purpose of making the TM learn different predictions for D and Y.

In [ ]:
tm = TM(
    columnDimensions = (2048,),
    cellsPerColumn=8,
    initialPermanence=0.21,
    connectedPermanence=0.3,
    minThreshold=15,
    maxNewSynapseCount=40,
    permanenceIncrement=0.1,
    permanenceDecrement=0.1,
    activationThreshold=15,
    predictedSegmentDecrement=0.01,
)

for t in range(75):
    seq = random.choice([ seq1, seq2 ])
    for inp in seq:
        tm.compute(dataset[inp], learn=True)

We now have a look at the output of the TM when presented with the individual characters A, B, C, D, X, and Y.  
We should observe simultaneous predictions when presented with character D (predicting A and X), character Y (predicting A and X), and when presented with character C (predicting D and Y).

In [ ]:
showPredictions()